# NNLM (Neural Network Language Model)

In [ ]:
import torch
import torch.nn as nn               # 신경망 레이어/손실함수
import torch.optim as optim         # 옵티마이저

In [ ]:
# 컨텐스트 단어들을 임베딩 -> MLP로 변환해 다음 단어 분포를 예측하는 NNLM 모델
class NNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, context_size):
        super(NNLM, self).__init__()    # nn.Module 초기화
        self.embed = nn.Embedding(vocab_size, embed_size)   # 단어 ID -> 임베딩 벡터
        self.fc1 = nn.Linear(context_size * embed_size, hidden_size)    #(컨텍스트 임베딩 연결) -> 은닉층
        self.relu = nn.ReLU()                               # 비선형 활성화 함수
        self.fc2 = nn.Linear(hidden_size, vocab_size)       # 은닉층 -> 단어 분포 로짓(어휘 크기)
        self.log_softmax = nn.LogSoftmax(dim=1)             # 로짓 -> 로그 확률(dim-1 : 배치 기준)

    def forward(self, x):
        embeds = self.embed(x)      # (B, context_size) -> (B, context_size, embed_size)
        embeds = embeds.view(embeds.size(0), -1)    # (B, context_size * embed_size)로 평탄화(연결)
        output = self.fc1(embeds)   # 은닉층 선형 변환
        output = self.relu(output)  # 비선형성 추가
        output = self.fc2(output)   # 어휘 크기만큼 로짓 출력
        log_probs = self.log_softmax(output)    # 로짓을 로그 확률 분포로 변환
        return log_probs            # (B,vocab_size) 반환

In [ ]:
VOCAB_SIZE = 5000       # 어휘 단어 수
EMBED_SIZE = 300        # 임베딩 벡터 차원
HIDDEN_SIZE = 128       # 은닉층 뉴런 개수
CONTEXT_SIZE = 2        # 컨테스트 단어 개수(입력으로 넣는 이전 단어 수)

model = NNLM(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, CONTEXT_SIZE)
print(model)

NNLM(
  (embed): Embedding(5000, 300)
  (fc1): Linear(in_features=600, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=5000, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [ ]:
# dummy data 생성 (X데이터는 컨텍스트 단어 ID 행렬 8x2, y는 각 샘플의 정답 던어 ID 벡터 (8개) 생성)
X = torch.randint(0, VOCAB_SIZE, (8, CONTEXT_SIZE)) # (배치 8, 컨텍스트2) 범위 내 단어 ID 랜덤 생성
y = torch.randint(0, VOCAB_SIZE, (8,))              # (배치 8,) 다음 단어(정답) ID를 랜덤 생성

In [6]:
X

tensor([[2553,  330],
        [ 314,  489],
        [4918, 1978],
        [2224,  852],
        [1809, 2196],
        [3072, 4466],
        [ 564, 4883],
        [1040, 3840]])

In [7]:
y

tensor([2625,  771, 1176, 1006, 2079,  438,  943, 1326])

In [ ]:
criterion = nn.NLLLoss()    # 로그확률(log_softmax 출력)용 손실함수(Negative Log Likehood)
optimizer = optim.Adam(model.parameters(), lr=0.001)    # 파라미터 업데이트 설정

model.train()                   # 학습모드
optimizer.zero_grad()           # 이전 step 기울기 초기화
output = model(X)               # 순전파 : 로그확률 예측 (B, vocab_size)
loss = criterion(output, y)     # 손실계산
loss.backward()                 # 역전파 : 기울기 계산
optimizer.step()                # 계산된 기울기로 파라미터 업데이트

print(loss.item())              # 손실값 출력

8.543664932250977
